# Overview

The purpose of this analysis is to observe the number of claims and medications over time.<br>
Visit Data.gov to download the data used for this analysis.

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path
import matplotlib
from datetime import date
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

## Data ingest

In [2]:
# load data
bene_df = pd.read_csv('data/beneficiary_2022.csv', sep="|", usecols=['BENE_ID'], low_memory=False)
inpatient_df = pd.read_csv('data/inpatient.csv', sep="|", usecols=['CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD', 'CLM_LINE_NUM'], low_memory=False)
outpatient_df = pd.read_csv('data/outpatient.csv', sep="|", usecols=['CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD', 'CLM_LINE_NUM'], low_memory=False)
carrier_df = pd.read_csv('data/carrier.csv', sep="|", usecols=['CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD', 'LINE_NUM'], low_memory=False)
dme_df = pd.read_csv('data/dme.csv', sep="|", usecols=['CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD','LINE_NUM'],low_memory=False)
hha_df = pd.read_csv('data/hha.csv', sep="|", usecols=['CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT','HCPCS_CD','CLM_LINE_NUM'],low_memory=False)
hospice_df = pd.read_csv('data/hospice.csv', sep="|", usecols=['CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD', 'CLM_LINE_NUM'], low_memory=False)
snf_df = pd.read_csv('data/snf.csv', sep="|", usecols=['CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD', 'CLM_LINE_NUM'], low_memory=False)
pde_df = pd.read_csv('data/pde.csv', sep="|", usecols=['PDE_ID', 'BENE_ID', 'SRVC_DT', 'PROD_SRVC_ID'], low_memory=False)

In [3]:
# convert to datetime
inpatient_df['CLM_FROM_DT']=pd.to_datetime(inpatient_df.CLM_FROM_DT).dt.date
inpatient_df['CLM_THRU_DT']=pd.to_datetime(inpatient_df.CLM_THRU_DT).dt.date
outpatient_df['CLM_FROM_DT']=pd.to_datetime(outpatient_df.CLM_FROM_DT).dt.date
outpatient_df['CLM_THRU_DT']=pd.to_datetime(outpatient_df.CLM_THRU_DT).dt.date
carrier_df['CLM_FROM_DT']=pd.to_datetime(carrier_df.CLM_FROM_DT).dt.date
carrier_df['CLM_THRU_DT']=pd.to_datetime(carrier_df.CLM_THRU_DT).dt.date
dme_df['CLM_FROM_DT']=pd.to_datetime(dme_df.CLM_FROM_DT).dt.date
dme_df['CLM_THRU_DT']=pd.to_datetime(dme_df.CLM_THRU_DT).dt.date
hha_df['CLM_FROM_DT']=pd.to_datetime(hha_df.CLM_FROM_DT).dt.date
hha_df['CLM_THRU_DT']=pd.to_datetime(hha_df.CLM_THRU_DT).dt.date
hospice_df['CLM_FROM_DT']=pd.to_datetime(hospice_df.CLM_FROM_DT).dt.date
hospice_df['CLM_THRU_DT']=pd.to_datetime(hospice_df.CLM_THRU_DT).dt.date
snf_df['CLM_FROM_DT']=pd.to_datetime(snf_df.CLM_FROM_DT).dt.date
snf_df['CLM_THRU_DT']=pd.to_datetime(snf_df.CLM_THRU_DT).dt.date
pde_df['SRVC_DT']=pd.to_datetime(pde_df.SRVC_DT).dt.date
pde_df['SRVC_DT']=pd.to_datetime(pde_df.SRVC_DT).dt.date

### N synthetic beneficiaries 

In [4]:
# unique count of BENE_IDs
unique_ids = bene_df['BENE_ID'].unique()

# print results
print(f"{len(unique_ids)} unique ids and {bene_df.shape[0]} total rows")

8671 unique ids and 8671 total rows


### N total synthetic claims covering a period from T1 to T2 

In [5]:
df_list = [carrier_df, inpatient_df, outpatient_df, dme_df, snf_df, hospice_df, hha_df]

# initiate counter
counter0 = 0

# initiate clm_total
clm_total = 0

# for loop
for i in df_list:
    # iterate through list of dataframes
    temp_df = i
    # merge bene_df and all other dataframes in list on BENE_ID
    merged_temp_df = pd.merge(bene_df, temp_df, on ='BENE_ID', how ="left")
    # create new variable line_col and retrieve line_num variable from each dataframe; note line_num variable name varies across dataframes
    line_col = merged_temp_df.filter(regex='LINE_NUM', axis=1).columns.tolist()[0]
    # create df to retrieve specific variables only
    line_count_df = merged_temp_df.groupby([line_col, 'CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD']).size().sort_values(ascending=False).to_frame().reset_index().rename(columns={0: "tot_claims"})
    # count number of carrier claims
    clm_counts = line_count_df.shape[0]
    # append the total count of claims per dataframe
    clm_total += clm_counts
    # identify min and max claim dates
    min_clm_dt = line_count_df['CLM_FROM_DT'].min()
    max_clm_dt = line_count_df['CLM_THRU_DT'].max()
    # count distinct HCPCS_CD (procedures)
    procedures = len(line_count_df['HCPCS_CD'].unique())
    # increment counter to iterate through df_list    
    counter0+=1

# print results
print(f"{clm_total} total synthetic claims")

1175824 total synthetic claims


### Claims covering a period from T1 to T2 and M distinct procedures

In [6]:
# create list of dataframes
df_list = [carrier_df, inpatient_df, outpatient_df, dme_df, snf_df, hospice_df, hha_df]

# create list of claim types
claim_type = ["carrier", "inpatient", "outpatient", "dme", "snf", "hospice", "hha"]

# initiate counters
counter0 = 0
counter1 = 1

# for loop
for i in df_list:
    # iterate through list of dataframes
    temp_df = i
    # merge bene_df and all other dataframes in list on BENE_ID
    merged_temp_df = pd.merge(bene_df, temp_df, on ='BENE_ID', how ="inner")
    # create new variable line_col and retrieve line_num variable from each dataframe; note line_num variable name varies across dataframes
    line_col = merged_temp_df.filter(regex='LINE_NUM', axis=1).columns.tolist()[0]
    # create df to retrieve specific variables only
    line_count_df = merged_temp_df[[line_col, 'CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'HCPCS_CD']]
    # count number of claims
    clm_counts = line_count_df.shape[0]
    # identify min and max claim dates
    min_clm_dt = line_count_df['CLM_FROM_DT'].min()
    max_clm_dt = line_count_df['CLM_THRU_DT'].max()                                                                                     
    # count distinct HCPCS_CD (procedures)
    procedures = len(line_count_df['HCPCS_CD'].unique())
    # print results
    print(f"{clm_counts} {claim_type[counter0]} claims covering a period from {min_clm_dt} to {max_clm_dt} and {procedures} distinct procedures")
    # increment counter to iterate through df_list
    counter0+=1
    # increment counter to iterate through df_list and claim_type    
    counter1+=1

1120655 carrier claims covering a period from 2015-03-06 to 2023-03-02 and 38 distinct procedures
58030 inpatient claims covering a period from 2015-02-25 to 2023-03-02 and 106 distinct procedures
574861 outpatient claims covering a period from 2015-03-06 to 2023-03-02 and 107 distinct procedures
103798 dme claims covering a period from 2015-01-08 to 2023-03-02 and 40 distinct procedures
12526 snf claims covering a period from 2015-01-08 to 2023-03-01 and 27 distinct procedures
12088 hospice claims covering a period from 2014-11-18 to 2023-02-27 and 13 distinct procedures
6215 hha claims covering a period from 2015-03-09 to 2023-03-01 and 22 distinct procedures


### N Part D claims covering a period from T1 to T2 and M distinct medications
 

In [7]:
# merge bene_df and all other dataframes in list on BENE_ID
merged_temp_df = pd.merge(bene_df, pde_df, on ='BENE_ID', how ="left")

# create data frame consisting of line number,'CLM_ID','BENE_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', and 'HCPCS_CD'. HCPCS_CD represents procedures in this case
# line_count_df = pde_df.groupby(['PDE_ID', 'BENE_ID', 'SRVC_DT', 'PROD_SRVC_ID']).size().sort_values(ascending=False).to_frame().reset_index().rename(columns={0: "tot_claims"})
line_count_df = pde_df[['PDE_ID', 'BENE_ID', 'SRVC_DT', 'PROD_SRVC_ID']]

# count number of pde claims
pde_clm_counts = line_count_df.shape[0]

# identify min and max claim dates
min_clm_dt = line_count_df['SRVC_DT'].min()
max_clm_dt = line_count_df['SRVC_DT'].max()

# count distinct medications
medications = len(line_count_df['PROD_SRVC_ID'].unique())

# print results
print(f"{pde_clm_counts} pde claims covering a period from {min_clm_dt} to {max_clm_dt} and {medications} distinct medications")

515520 pde claims covering a period from 2015-03-08 to 2023-03-03 and 14627 distinct medications
